In [ ]:
import pandas as pd
import numpy as np
import time
import requests
import json

In [ ]:
# pd.read_csv('aggregated_features.csv')
# corrected_series_table.to_csv('cleaned_series_table.csv', index=False)